<a href="https://colab.research.google.com/github/yuguerten/British_Airways_Reviews/blob/master/British_Airways_Data_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**installing the libararies**

In [1]:
!pip install beautifulsoup4
!pip install requests

**Import libraries**

In [2]:
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt

**Scraping the 1st page**

In [3]:
url = "https://www.airlinequality.com/airline-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100"
res = requests.get(url)

In [4]:
print(res)

<Response [200]>


In [ ]:
soup = BeautifulSoup(res.text)
soup

In [ ]:
reviews = soup.select(".text_content , time")
reviews

In [11]:
date_review = []
review_text = []
for i in range(0, len(reviews), 4):
    review_text.append(reviews[i + 1].get_text(strip=True))
    date_review.append(reviews[i+2].get_text(strip=True))


for i in range(len(date_review)):
    print(f"Date review: {date_review[i]}")
    print(f"Full review: {review_text[i]}")
    print("\n")

Date review: 21st December 2023
Full review: ✅Trip Verified| This was our first flight with British Airways in over 13 years and as usual I couldn’t fault them. As we were connecting, we had little interaction with ground staff - staff in the lounge were friendly and helpful. Cabin crew on the flight were great - very welcoming and friendly. Our meals were delicious & no shortage of drinks being offered. The seats were economy seats with the middle row blocked off and some recline. We arrived into Stockholm a little early through a snow shower which was a nice welcome back to Stockholm. Bags (checked in at JFK) were delivered safely within 20 mins of arriving.


Date review: 20th December 2023
Full review: Not Verified|  Beware! BA don't provide any refund due to a very serious medical reason. My father was diagnosed with a severe heart defect a couple of weeks before his flight date and BA refused to give any refund as this was not a good enough reason!! We sent them a medical note fr

In [12]:
len(review_text)

50

**Scarpping all the pages**

In [27]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways/page/{}/?sortby=post_date%3ADesc&pagesize=100"

review_text = []
date_review = []
number_of_pages = 38

for page_number in range(1, number_of_pages + 1):
    url = base_url.format(page_number)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Select reviews for the current page
    reviews = soup.select(".text_content, time")

    print("Number of reviews on page", page_number, ":", len(reviews))

    for i in range(0, len(reviews), 4):
        # Make sure the index is within the bounds of the reviews list
        if i + 2 < len(reviews):
            review_text.append(reviews[i + 1].get_text(strip=True))
            date_review.append(reviews[i + 2].get_text(strip=True))
        else:
            print("Warning: Skipping incomplete review on page", page_number)

Number of reviews on page 1 : 200
Number of reviews on page 2 : 200
Number of reviews on page 3 : 200
Number of reviews on page 4 : 200
Number of reviews on page 5 : 200
Number of reviews on page 6 : 200
Number of reviews on page 7 : 200
Number of reviews on page 8 : 200
Number of reviews on page 9 : 200
Number of reviews on page 10 : 200
Number of reviews on page 11 : 200
Number of reviews on page 12 : 200
Number of reviews on page 13 : 200
Number of reviews on page 14 : 200
Number of reviews on page 15 : 200
Number of reviews on page 16 : 200
Number of reviews on page 17 : 200
Number of reviews on page 18 : 200
Number of reviews on page 19 : 200
Number of reviews on page 20 : 200
Number of reviews on page 21 : 200
Number of reviews on page 22 : 200
Number of reviews on page 23 : 200
Number of reviews on page 24 : 200
Number of reviews on page 25 : 200
Number of reviews on page 26 : 200
Number of reviews on page 27 : 200
Number of reviews on page 28 : 200
Number of reviews on page 29 

In [28]:
len(review_text)

1860

In [29]:
len(date_review)

1860

In [30]:
import pandas as pd

In [33]:
British_Airways_reviews = {
    'Review_Date' : date_review,
    'Review' : review_text,
}
data = pd.DataFrame(British_Airways_reviews)

In [37]:
data

,Review_Date,Review
0,21st December 2023,✅Trip Verified| This was our first flight with...
1,20th December 2023,Not Verified| Beware! BA don't provide any re...
2,17th December 2023,Not Verified| All 4 of our flights were fine i...
3,14th December 2023,✅Trip Verified| I was going to fly to Frankfu...
4,12th December 2023,✅Trip Verified| Once again a terrible business...
...,...,...
1855,29th August 2012,LHR-HKG on Boeing 747 - 23/08/12. Much has bee...
1856,29th August 2012,LHR-JFK-LAX-LHR. Check in was ok apart from be...
1857,29th August 2012,YYZ to LHR - July 2012 - I flew overnight in p...
1858,28th August 2012,Flew LHR - VIE return operated by bmi but BA a...


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Date  1860 non-null   object
 1   Review       1860 non-null   object
dtypes: object(2)
memory usage: 29.2+ KB


In [36]:
len(data['Review'].unique())

1858

In [38]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = text.replace('✅Trip Verified|', '')
    text = text.replace('✅verified review', '')
    text = text.replace('✅trip verified', '')
    text = text.replace('Not Verified|','')
    return text

round1 = lambda x: clean_text_round1(x)

In [39]:
data_round1 = pd.DataFrame(data['Review'].apply(round1))

In [40]:
data_round1

,Review
0,this was our first flight with british airway...
1,not verified beware ba dont provide any refun...
2,not verified all of our flights were fine in ...
3,i was going to fly to frankfurt from glasgow...
4,once again a terrible business class experien...
...,...
1855,lhrhkg on boeing much has been written abou...
1856,lhrjfklaxlhr check in was ok apart from being ...
1857,yyz to lhr july i flew overnight in premium...
1858,flew lhr vie return operated by bmi but ba ai...


In [41]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [42]:
British_Airways_reviews = pd.DataFrame(data_round1['Review'].apply(round2))

In [46]:
British_Airways_reviews

,Review
0,this was our first flight with british airway...
1,not verified beware ba dont provide any refun...
2,not verified all of our flights were fine in ...
3,i was going to fly to frankfurt from glasgow...
4,once again a terrible business class experien...
...,...
1855,lhrhkg on boeing much has been written abou...
1856,lhrjfklaxlhr check in was ok apart from being ...
1857,yyz to lhr july i flew overnight in premium...
1858,flew lhr vie return operated by bmi but ba ai...


In [47]:
British_Airways_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1860 non-null   object
dtypes: object(1)
memory usage: 14.7+ KB


In [48]:
my_data = British_Airways_reviews.to_csv(' British_Airways_reviews.csv', index=False)